In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('ML 08_Smoking Data.csv')
df.head()

,region,age,sex,bmi,children,smoker
0,southwest,19,female,27.900,0,yes
1,southeast,18,male,33.770,1,no
2,southeast,28,male,33.000,3,no
3,northwest,33,male,22.705,0,no
4,northwest,32,male,28.880,0,no


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   region    1338 non-null   object 
 1   age       1338 non-null   int64  
 2   sex       1338 non-null   object 
 3   bmi       1338 non-null   float64
 4   children  1338 non-null   int64  
 5   smoker    1338 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 62.8+ KB


# Encoding (using Label Encoder)

In [3]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [4]:
df.region=le.fit_transform(df.region)
df.sex=le.fit_transform(df.sex)
df.smoker=le.fit_transform(df.smoker)
df.head()

,region,age,sex,bmi,children,smoker
0,3,19,0,27.900,0,1
1,2,18,1,33.770,1,0
2,2,28,1,33.000,3,0
3,1,33,1,22.705,0,0
4,1,32,1,28.880,0,0


# Split the Data 

In [5]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df.drop('smoker',axis=1),df['smoker'],train_size=.70,random_state=42)

# Apply Decision Tree Classification 

In [6]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(xtrain,ytrain)
dtc.score(xtest,ytest)   # mean accuracy

0.6666666666666666

# Apply Random Forest Classifier ML Model

In [7]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(xtrain,ytrain)
rfc.score(xtest,ytest)

0.7860696517412935

# Hyper Parameter Tuning (of Random Forest Classifier ML Model)

# Using Gride Search CV

In [8]:
trees=np.random.randint(25,150,30)
criterion=['gini','entropy']
max_depth=np.random.randint(1,15,20)
min_sample_split=[2,3,4]
max_features=['sqrt' , 'log2']

In [9]:
# param_grid
ids={
    'n_estimators' : trees,
    'criterion' : criterion,
    'max_depth' : max_depth,
    'min_samples_split' : min_sample_split,
    'max_features' : max_features
}

In [10]:
from sklearn.model_selection import GridSearchCV
gscv=GridSearchCV(rfc,ids,cv=3)

In [11]:
gscv.fit(xtrain,ytrain)
cv_result=pd.DataFrame(gscv.cv_results_)

In [12]:
cv_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.206777,0.016758,0.020858,0.003834,gini,7,sqrt,2,107,"{'criterion': 'gini', 'max_depth': 7, 'max_fea...",0.791667,0.782051,0.788462,0.787393,0.003997,2823
1,0.136700,0.014220,0.014181,0.001570,gini,7,sqrt,2,77,"{'criterion': 'gini', 'max_depth': 7, 'max_fea...",0.791667,0.782051,0.788462,0.787393,0.003997,2823
2,0.074549,0.007853,0.010414,0.007364,gini,7,sqrt,2,47,"{'criterion': 'gini', 'max_depth': 7, 'max_fea...",0.788462,0.772436,0.788462,0.783120,0.007555,3834
3,0.055090,0.001813,0.007336,0.000957,gini,7,sqrt,2,26,"{'criterion': 'gini', 'max_depth': 7, 'max_fea...",0.788462,0.785256,0.782051,0.785256,0.002617,3415
4,0.146609,0.012798,0.013961,0.000841,gini,7,sqrt,2,79,"{'criterion': 'gini', 'max_depth': 7, 'max_fea...",0.788462,0.782051,0.791667,0.787393,0.003997,2823


In [13]:
gscv.best_score_

0.7927350427350427

In [14]:
gscv.best_params_

{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'sqrt',
 'min_samples_split': 2,
 'n_estimators': 59}

In [25]:
rfc2=gscv.best_estimator_

In [16]:
rfc2.fit(xtrain, ytrain)
rfc2.score(xtest, ytest)

0.8034825870646766